In [1]:
%%writefile requirements.txt
mmh3
bitarray
blist
countmemaybe

Overwriting requirements.txt


In [2]:
# !pip install -r requirements.txt

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
n_times = 3
size_list = list(map(int, [1e2, 1e3, 1e4, 1e5, 1e6]))

In [5]:
%%writefile morris_counter.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import math
import array
import random


class MorrisCounter(object):

    def __init__(self, array_type=u'B', nbr_counters=1):
        self.exponents = array.array(array_type, [0] * nbr_counters)

    def __len__(self):
        return len(self.exponents)

    def add_counter(self):
        """Add a new zeroed counter"""
        self.exponents.append(0)

    def get(self, counter=0):
        """Calculate approximate value represented by counter"""
        return math.pow(2, self.exponents[counter])

    def add(self, counter=0):
        """Probabilistically add 1 to counter"""
        value = self.get(counter)
        probability = 1.0 / value
        if random.uniform(0, 1) < probability:
            self.exponents[counter] += 1
    
    def __str__(self):
        return str(self.get())
    
    def __repr__(self):
        return self.get()
        

Overwriting morris_counter.py


In [6]:
import morris_counter

cnt = morris_counter.MorrisCounter()
for n in range(50000):
    cnt.add()
print(cnt)

65536.0


In [7]:
import morris_counter

# compare actual size and counter size
cnt = morris_counter.MorrisCounter()
for size in size_list:
    for __ in range(size): 
        cnt.add()

    print("actual: {} Counter: {} ({:.2f}% error)".format(
        size, int(cnt.get()), abs(cnt.get() - size) / size * 100))

actual: 100 Counter: 64 (36.00% error)
actual: 1000 Counter: 1024 (2.40% error)
actual: 10000 Counter: 8192 (18.08% error)
actual: 100000 Counter: 65536 (34.46% error)
actual: 1000000 Counter: 524288 (47.57% error)


In [8]:
import timeit

# estimate time
for size in size_list:
    time = timeit.Timer("for __ in range({}): cnt.add()".format(size),
                        "from morris_counter import MorrisCounter;"
                        "cnt = MorrisCounter()")
    lookup_time = time.timeit(n_times) / n_times   
    print("size: {} time: {:.2f}".format(size, lookup_time))


size: 100 time: 0.00
size: 1000 time: 0.00
size: 10000 time: 0.01
size: 100000 time: 0.09
size: 1000000 time: 0.85


# KMinValues

In [9]:
%%writefile kmin_values.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import mmh3
import blist


class KMinValues(object):

    def __init__(self, num_hashes):
        self.num_hashes = num_hashes
        self.data = blist.sortedset()

    def add(self, item):
        item_hash = mmh3.hash(item)
        self.data.add(item_hash)
        if len(self.data) > self.num_hashes:
            self.data.pop()

    def __len__(self):
        if len(self.data) < self.num_hashes:
            return len(self.data)
        value = (self.num_hashes - 1) * (2 ** 32 - 1) / \
            float(self.data[-2] + 2 ** 31 - 1)
        return int(value)


Overwriting kmin_values.py


In [10]:
import kmin_values

# add 10000 elements
kmv = kmin_values.KMinValues(50)
for x in list(range(0, 10000)):
    kmv.add(str(x))
    
len(kmv)

9920

In [11]:
import kmin_values

# add 10000 dublicated elements
kmv = kmin_values.KMinValues(50)
for x in 2 * list(range(0, 10000)):
    kmv.add(str(x))
    
len(kmv)

9920

In [12]:
import kmin_values

# compare actual size and counter size
kmv = kmin_values.KMinValues(1024)
for size in size_list:
    for i in range(size): 
        kmv.add(str(i))

    print("actual: {} KMinValues: {} ({:.2f}% error)".format(
        size, len(kmv), abs(len(kmv) - size) / size * 100))

actual: 100 KMinValues: 100 (0.00% error)
actual: 1000 KMinValues: 1000 (0.00% error)
actual: 10000 KMinValues: 10078 (0.78% error)
actual: 100000 KMinValues: 101311 (1.31% error)
actual: 1000000 KMinValues: 1009457 (0.95% error)


In [13]:
import timeit

for size in size_list:
    time = timeit.Timer("for x in range({}): kmv.add(str(x))".format(size),
                        "from kmin_values import KMinValues;"
                        "kmv = KMinValues(3)")
    lookup_time = time.timeit(n_times) / n_times   
    print("size: {} time: {:.2f}".format(size, lookup_time))
    

size: 100 time: 0.00
size: 1000 time: 0.01
size: 10000 time: 0.07
size: 100000 time: 0.64
size: 1000000 time: 6.47


# KMinValues from countmemaybe


In [14]:
import countmemaybe

# add 10000 elements
kmv = countmemaybe.KMinValues(k=50)
for x in list(range(0, 10000)):
    kmv.add(str(x))
    
kmv.cardinality()

9176.640274793317

In [15]:
import countmemaybe

# compare actual size and counter size
kmv = countmemaybe.KMinValues(k=1024)
for size in size_list:
    for i in range(size): 
        kmv.add(str(i))

    print("actual: {} KMinValues: {} ({:.2f}% error)".format(
        size, int(kmv.cardinality()), abs(kmv.cardinality() - size) / size * 100))

actual: 100 KMinValues: 100 (0.00% error)
actual: 1000 KMinValues: 1000 (0.00% error)
actual: 10000 KMinValues: 10292 (2.93% error)
actual: 100000 KMinValues: 97765 (2.23% error)
actual: 1000000 KMinValues: 993007 (0.70% error)


In [16]:
import countmemaybe

# add 10000 dublicated elements
kmv = countmemaybe.KMinValues(k=50)
for x in 2 * list(range(0, 10000)):
    kmv.add(str(x))
    
kmv.cardinality()

9176.640274793317

In [17]:
import countmemaybe

# add one element
kmv = countmemaybe.KMinValues(k=50)
for x in range(0, 10000):
    kmv.add('1')
    
kmv.cardinality()

1

In [18]:
import countmemaybe

kmv1 = countmemaybe.KMinValues(k=50)
for x in range(0, 10000):
    kmv1.add(str(x))

print('kmv1', kmv1.cardinality())

kmv2 = countmemaybe.KMinValues(k=50)
for x in range(5000, 15000):
    kmv2.add(str(x))

print('kmv2', kmv2.cardinality())

kmv1 9176.640274793317
kmv2 8652.692389314136


In [19]:
print("""intersection : {} (true: {})
union        : {} (true: {})""".format(int(kmv1.cardinality_intersection(kmv2)), 5000,
                              int(kmv1.cardinality_union(kmv2)), 15000))

intersection : 6351 (true: 5000)
union        : 13808 (true: 15000)


In [20]:
import timeit

for size in size_list:
    time = timeit.Timer("for x in range({}): kmv.add(str(x))".format(size),
                        "from countmemaybe.kminvalues import KMinValues;"
                        "kmv = KMinValues(k=3)")
    lookup_time = time.timeit(n_times) / n_times   
    print("size: {} time: {:.2f}".format(size, lookup_time))


size: 100 time: 0.00
size: 1000 time: 0.00
size: 10000 time: 0.02
size: 100000 time: 0.17
size: 1000000 time: 1.68


# BloomFilter

In [21]:
%%writefile bloom_filter.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import bitarray
import math
import mmh3
import random


class BloomFilter(object):

    def __init__(self, capacity, error=0.005):
        """
        Initialize a bloom filter with given capacity and false positive rate
        """
        self.capacity = capacity
        self.error = error
        self.num_bits = int(-capacity * math.log(error) / math.log(2) ** 2) + 1
        self.num_hashes = int(self.num_bits * math.log(2) / capacity) + 1
        self.data = bitarray.bitarray(self.num_bits)

    def _indexes(self, key):
        h1, h2 = mmh3.hash64(key)
        for i in range(self.num_hashes):
            yield (h1 + i * h2) % self.num_bits

    def add(self, key):
        for index in self._indexes(key):
            self.data[index] = True

    def __contains__(self, key):
        return all(self.data[index] for index in self._indexes(key))

    def _len(self):
        num_bits_on = self.data.count(True)
        if num_bits_on == self.num_bits:
            raise Exception('bloom filter is full')
        log = 1.0 - num_bits_on / self.num_bits
        return -1.0 * self.num_bits * math.log(log) / self.num_hashes
    
    def __len__(self):
        return int(self._len())
    
    def get_free_bits_ratio(self):
        return self.data.count(True) / self.num_bits

    @staticmethod
    def union(bloom_a, bloom_b):
        assert bloom_a.capacity == bloom_b.capacity, "Capacities must be equal"
        assert bloom_a.error == bloom_b.error, "Error rates must be equal"

        bloom_union = BloomFilter(bloom_a.capacity, bloom_a.error)
        bloom_union.data = bloom_a.data | bloom_b.data
        return bloom_union
    

Overwriting bloom_filter.py


In [22]:
%%writefile tools.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import random


def confusion_matrix(bloomfilter, size, low=0, prob=0.5, random_state=None):
        
    random.seed(random_state)
    num_list = set(map(str, range(low, low + size)))
    seen_list = {x for x in num_list if random.random() < prob}
    
    for x in seen_list:
        bloomfilter.add(x)
        
    true = (x in seen_list for x in num_list)
    pred = (x in bloomfilter for x in num_list)

    results = {}
    for t, p in [(True, True), (True, False), (False, True), (False, False)]:
        results[(t, p)] = 0

    for t, p in zip(true, pred):
        results[(t, p)] += 1
        
    return results, bloomfilter

def accuracy(results):
    return (results[(True, True)] + results[False, False]) / (sum(results.values()))

def precision(results):
    return results[(True, True)] / (results[(True, True)] + results[False, True])

def recall(results):
    return results[(True, True)] / (results[(True, True)] + results[True, False])

def pretty_print(results):
    print('          True     False')
    print('Positive  {:<8d} {:<8d}'.format(results[(True, True)], results[(False, True)]))
    print('Negative  {:<8d} {:<8d}'.format(results[(True, False)], results[(False, False)]))
    print('accuracy  : {:.4f}'.format(accuracy(results)))
    print('precision : {:.4f}'.format(precision(results)))
    print('recall    : {:.4f}'.format(recall(results)))


Overwriting tools.py


In [23]:
import bloom_filter

# set Bloom Filter with capacity 1024 elements and error rate 0.5%
blf = bloom_filter.BloomFilter(capacity=1024, error=0.005)
# add 1000 elements
for x in range(0, 1000):
    blf.add(str(x))

# check 2000 elements
sum_exists = sum((1 for x in range(0, 2000) if str(x) in blf))
error_rate = abs(sum_exists - 1000) / 1000 * 100
print("total elements : {} (error rate {}%)".format(sum_exists, error_rate))


total elements : 1024 (error rate 2.4%)


In [24]:
import bloom_filter
from tools import confusion_matrix, pretty_print

blf = bloom_filter.BloomFilter(capacity=1024, error=0.005)

# check quality of bloom filter using confusion matrix
results, blf = confusion_matrix(blf, size=2000, random_state=47)
pretty_print(results)

          True     False
Positive  988      26      
Negative  0        986     
accuracy  : 0.9870
precision : 0.9744
recall    : 1.0000


In [25]:
import bloom_filter
from tools import confusion_matrix, pretty_print

capacity = 4000
for size in [100, 1000, 10000, 20000, 40000]:
    print('size      : {}'.format(size))
    results, blf = confusion_matrix(
        bloom_filter.BloomFilter(capacity), size=size, random_state=47)
    print('free bits : {:.4f}'.format(blf.get_free_bits_ratio()))
    pretty_print(results)
    print()

size      : 100
free bits : 0.2826
          True     False
Positive  49       0       
Negative  0        51      
accuracy  : 1.0000
precision : 1.0000
recall    : 1.0000

size      : 1000
free bits : 0.1873
          True     False
Positive  479      0       
Negative  0        521     
accuracy  : 1.0000
precision : 1.0000
recall    : 1.0000

size      : 10000
free bits : 0.7315
          True     False
Positive  4920     370     
Negative  0        4710    
accuracy  : 0.9630
precision : 0.9301
recall    : 1.0000

size      : 20000
free bits : 0.8789
          True     False
Positive  9954     3534    
Negative  0        6512    
accuracy  : 0.8233
precision : 0.7380
recall    : 1.0000

size      : 40000
free bits : 0.9891
          True     False
Positive  19784    18732   
Negative  0        1484    
accuracy  : 0.5317
precision : 0.5137
recall    : 1.0000



In [26]:
import timeit

for size in size_list:
    time = timeit.Timer("for x in range({}): blf.add(str(x))".format(size),
                        "from bloom_filter import BloomFilter;"
                        "blf = BloomFilter({})".format(size))
    lookup_time = time.timeit(n_times) / n_times   
    print("size: {} time: {:.2f}".format(size, lookup_time))


size: 100 time: 0.00
size: 1000 time: 0.01
size: 10000 time: 0.05
size: 100000 time: 0.52
size: 1000000 time: 5.28


# HyperLogLog

In [27]:
import countmemaybe

# add 10000 elements
hll = countmemaybe.HyperLogLog(b=4)
for x in list(range(0, 10000)):
    hll.add(str(x))
    
hll.cardinality()

9504.26461952862

In [28]:
import countmemaybe

hll1 = countmemaybe.HyperLogLog()
for x in range(0, 10000):
    hll1.add(str(x))

print('hll1', hll1.cardinality())

hll2 = countmemaybe.HyperLogLog()
for x in range(5000, 15000):
    hll2.add(str(x))

print('hll2', hll2.cardinality())

hll1 9504.26461952862
hll2 8013.531843860895


In [29]:
print("""intersection : {} (true: {})
union        : {} (true: {})""".format(int(hll1.cardinality_intersection(hll2)), 5000,
                              int(hll1.cardinality_union(hll2)), 15000))

intersection : 1770 (true: 5000)
union        : 15747 (true: 15000)


In [30]:
import timeit

for size in size_list:
    time = timeit.Timer("for x in range({}): blf.add(str(x))".format(size),
                        "from countmemaybe import HyperLogLog;"
                        "blf = HyperLogLog()".format(size))
    lookup_time = time.timeit(n_times) / n_times   
    print("size: {} time: {:.2f}".format(size, lookup_time))


size: 100 time: 0.00
size: 1000 time: 0.00
size: 10000 time: 0.02
size: 100000 time: 0.17
size: 1000000 time: 1.82
